In [1]:
# import
import numpy as np
import pandas as pd
import os.path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score
import os
import math
# label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================

def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type   

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            if label != "":
                dataset.append(
                                {   
                                    'uuid': root.split("\\")[-1],
                                    'label': label,
                                    'image_path': os.path.join(root, file)
                                }
                            )

    return dataset

In [2]:
## 參數設置
image_dir = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\front"
concat_type = "concat2"
class_count = 3

In [3]:
## load data and  labels
# =========================
data = load_path(image_dir, class_count)
labels = []
filepaths = []
for row in data:
    labels.append(row['label'])
    filepaths.append(row['image_path'])

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([filepaths, labels], axis=1)
# =========================


In [ ]:
## split image
# =========================
train_df_front, test_df_front = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])
print("Training set label distribution:\n", train_df_front['Label'].value_counts(normalize=False))
print("Test set label distribution:\n", test_df_front['Label'].value_counts(normalize=False))
# =========================

preprocessing_function_chosen_front = tf.keras.applications.resnet50.preprocess_input
# preprocessing_function_chosen_side = tf.keras.applications.efficientnet.preprocess_input
preprocessing_function_chosen_side = tf.keras.applications.resnet50.preprocess_input

# front images
# =========================
train_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                    preprocessing_function=preprocessing_function_chosen_front,
                                                                    validation_split=0.2)
test_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_front)

train_images_front = train_generator_front.flow_from_dataframe(
    dataframe=train_df_front,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False,
    seed=42,
    subset='training'
)

val_images_front = train_generator_front.flow_from_dataframe(
    dataframe=train_df_front,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False,
    seed=42,
    subset='validation'
)

test_images_front = test_generator_front.flow_from_dataframe(
    dataframe=test_df_front,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)
# =========================


# side images
# =========================
train_df_side = train_df_front.copy()
test_df_side = test_df_front.copy()
train_df_side.loc[:, "Filepath"] = train_df_front["Filepath"].str.replace("front", "side")
test_df_side.loc[:, "Filepath"] = test_df_side["Filepath"].str.replace("front", "side")

train_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                    preprocessing_function=preprocessing_function_chosen_side,
                                                                    validation_split=0.2)
test_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_side)

train_images_side = train_generator_side.flow_from_dataframe(
    dataframe=train_df_side,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False,
    seed=42,
    subset='training'
)

val_images_side = train_generator_side.flow_from_dataframe(
    dataframe=train_df_side,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False,
    seed=42,
    subset='validation'
)

test_images_side = test_generator_side.flow_from_dataframe(
    dataframe=test_df_side,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)
# =========================


Training set label distribution:
 0    128
2     95
1     93
Name: Label, dtype: int64
Test set label distribution:
 0    32
2    24
1    24
Name: Label, dtype: int64
Found 253 validated image filenames belonging to 3 classes.
Found 63 validated image filenames belonging to 3 classes.
Found 80 validated image filenames belonging to 3 classes.
Found 253 validated image filenames belonging to 3 classes.
Found 63 validated image filenames belonging to 3 classes.
Found 80 validated image filenames belonging to 3 classes.


In [5]:
# test_df_side
# test_df_front


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Dropout, Add
from tensorflow.keras.applications import ResNet50



def create_front_extract():
    pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model._name = 'AP_pretrain_resnet_model'
    pretrained_model.trainable = False
    return pretrained_model

def create_side_extract():
    pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model._name = 'Lateral_pretrain_efficientnet_model'
    pretrained_model.trainable = False
    return pretrained_model

# input 
input_front = Input(shape=(224, 224, 3), name="AP(Mortise)")
input_side = Input(shape=(224, 224, 3), name="Lateral")

# model
model_front = create_front_extract()
model_side = create_side_extract()

features_front = model_front(input_front)
features_side = model_side(input_side)

features_front_x = tf.keras.layers.Dense(50, activation='relu', name='AP_dense_50')(features_front)
features_front_x = tf.keras.layers.Dense(128, activation='relu', name='AP_dense_128')(features_front_x)

features_side_x = tf.keras.layers.Dense(50, activation='relu', name='Lateral_dense_50')(features_side)
features_side_x = tf.keras.layers.Dense(128, activation='relu', name='Lateral_dense_128')(features_side_x)

fused_features = Concatenate(name="feature_fusion")([features_front_x, features_side_x])
# fused_features = tf.keras.layers.Dense(50, activation='relu', name='fusion_dense_50')(fused_features)
fused_features = Dropout(0.1)(fused_features)

final_output = Dense(class_count, activation='sigmoid', name='output_layer')(fused_features)
multi_view_model = None
multi_view_model = Model(
    inputs=[input_front, input_side],
    outputs=final_output
)
multi_view_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

multi_view_model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
AP(Mortise) (InputLayer)        [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Lateral (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
AP_pretrain_resnet_model (Funct (None, 2048)         23587712    AP(Mortise)[0][0]                
__________________________________________________________________________________________________
Lateral_pretrain_efficientnet_m (None, 1280)         4049571     Lateral[0][0]                    
______________________________________________________________________________________________

In [7]:
def multi_input_generator(front_gen, side_gen):
    while True:
        front_batch, y1 = next(front_gen)
        side_batch, y2 = next(side_gen)
        assert (y1 == y2).all(), "Label mismatch!"  # 確保標籤一致
        yield ([front_batch, side_batch], y1)

train_generator = multi_input_generator(train_images_front, train_images_side)
val_generator = multi_input_generator(val_images_front, val_images_side)

In [ ]:
## compile and evaluate
# =========================

print("-------Training " + "_" + concat_type + "-------")
batch_size = 64
## early stop 
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
history=model.fit([train_images_front, train_images_side], validation_data=[val_images_front, val_images_side], callbacks=[early_stopping], epochs=30)
## no early stop
# history = multi_view_model.fit(train_generator, validation_data=val_generator, epochs=30,
#                             steps_per_epoch= math.ceil(train_images_front.samples / batch_size), 
#                             validation_steps= math.ceil(val_images_front.samples / batch_size))


-------Training _concat2-------
Epoch 1/30
4/4 [==============================] - 9s 1s/step - loss: 1.1708 - accuracy: 0.2609 - val_loss: 1.1078 - val_accuracy: 0.3810
Epoch 2/30
4/4 [==============================] - 2s 658ms/step - loss: 1.0978 - accuracy: 0.4032 - val_loss: 1.0445 - val_accuracy: 0.4762
Epoch 3/30
4/4 [==============================] - 2s 615ms/step - loss: 1.0122 - accuracy: 0.5099 - val_loss: 0.9879 - val_accuracy: 0.5397
Epoch 4/30
4/4 [==============================] - 2s 617ms/step - loss: 0.9800 - accuracy: 0.5296 - val_loss: 0.9370 - val_accuracy: 0.6032
Epoch 5/30
4/4 [==============================] - 2s 604ms/step - loss: 0.9072 - accuracy: 0.6087 - val_loss: 0.8936 - val_accuracy: 0.6190
Epoch 6/30
4/4 [==============================] - 2s 599ms/step - loss: 0.8716 - accuracy: 0.6522 - val_loss: 0.8549 - val_accuracy: 0.6508
Epoch 7/30
4/4 [==============================] - 2s 601ms/step - loss: 0.8224 - accuracy: 0.7036 - val_loss: 0.8221 - val_accuracy

In [9]:
## save model to this path
# =========================
multi_view_model.save("./weights/"+concat_type+"_" + "_frac.h5")
# =========================


## print results
# =========================
test_generator = multi_input_generator(test_images_front, test_images_side)

batch_size=32
pred = multi_view_model.predict(test_generator,  steps=math.ceil(test_images_front.samples / batch_size))
predicted_labels = np.argmax(pred, axis=1)
# =========================



# create plots for accuracy and save it
# =========================
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
figAcc = plt.gcf()
my_file = os.path.join("./plots/"+concat_type+"_Accuracy.jpeg")
figAcc.savefig(my_file)
plt.clf()
# =========================


## create plots for loss and save it
# =========================
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
figAcc = plt.gcf()
my_file = os.path.join("./plots/"+concat_type+"_"+"_Loss.jpeg")
figAcc.savefig(my_file)
plt.clf()
# =========================


## plot confusion matrix
# =========================
if class_count == 2:
    display_labels = [0, 1]
elif class_count == 3:
    display_labels = [0, 1, 2]
elif class_count == 4:
    display_labels = [0, 1, 2, 3]


cm = confusion_matrix(test_images_front.labels, predicted_labels)
cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = display_labels)
cm_display.plot()
plt.title('Confusion Matrix')
figAcc = plt.gcf()
my_file = os.path.join("./plots/"+concat_type+"_"+"_Confusion Matrix.jpeg")
figAcc.savefig(my_file)
plt.clf()


d:\anaconda3\envs\bone_20240719\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>